# U.S. Medical Insurance Costs

# 1. Look over your dataset

Download a zip file here with the necessary datasets and an empty Jupyter Notebook where you can write your code.

Open insurance.csv and take a look at the file. Take note of how information is organized. How will this affect how you analyze the data in Python? Is there anything of particular interest to you in the dataset that you want to investigate? Think about these things before you jump into analyzing it.


* The data in the csv file is organized as a table: information of each person is stored in a row with 7 attributes in the columns:

    age, sex, bmi, children, smoker, region, charges
    
* Some of the columns are categorical (sex, smoker, region) while other are numerical (age: int, bmi: float, children: int, charges: float), all numbers are positive.
    
* There is no missing data

A library reading .csv files will be useful for importing the data. It would make sense to store the data in a dictionary with keys named according to the columns (variables) and values containing the list of all observations for each variable.

It would be intersting to know how much each of the 6 features - age, sex, BMI, number of children, whether the person smokes and region of residence - affects the charges (how much does the person pay for the medical insurance). But it is out of scope for now, we could go back to this question after we covered visualization and prediction (linear regression).


# 2. Scoping Your Project

Now that you have looked over your dataset, plan out what you want to analyze. What is it that you want to find out about this dataset? Based on the way information is organized, certain inspections may be easier to perform than others. As you map out the process, consider the scope of your analysis as well.

Properly scoping your project will greatly benefit you; scoping creates structure while requiring you to think through your entire project before you begin. You should start by stating the goals for your project, then gathering the data, and considering the analytical steps required. A proper project scope can be a great road map for your project, but keep in mind that some down-stream tasks may become dead ends which will require adjustment to the scope. 

We want to analyze the population itself and answer the following questions:
* Are people more likely to be healthy (no smoking, normal BMI) at younger age or older, with children or without them, if they are male or female?

    * Children vs. smoker - Are people more likely to be smokers with children or not?
    * Sex vs. smoker - Are females more likely to be smokers than males?
    * Age vs. smoker - which part of the population smokes the most? At what age do they usually start?
    * BMI vs. age - How does BMI distribution looks like for different age groups? Does BMI increase with age?
    * BMI vs. sex - How does BMI distribution looks like for males and for females? Which sex is more predisposed to obesity?
    * BMI vs. smoker - Do smokers have a lower BMI than non-smokers?
    * BMI vs. children - Does BMI increase with the number of children?
    
Since BMI and age variables are numerical, we should subdivide them into a few categories to simplify the analysis (since we decided to investigate the data without plotting).

# 3. Import your dataset

Import insurance.csv into your Python file and inspect the contents.


# 4. Save your dataset via Python variables

Save the features of your dataset (the columns) from insurance.csv by storing them in variables that can be used for analysis. As you consider what types of variables to use and how many you plan to create, think ahead about the parameters you wish to investigate and how your organization will impact this analysis.

# 5. Build out analysis functions or class methods

You now have everything you need to begin your analysis. You have organized the information from insurance.csv and have spent some time thinking about what it is you would like to investigate.

Now is the time to build out how you perform these investigations. Use the Python fundamentals you have learned so far to accomplish these tasks. There are many different ways you can achieve these analyses. In our hint, we will provide some ideas for how you can use Python to analyze data.

# 6. Project Extensions

You’re welcome to expand your analysis beyond what you have already done! Some potential extra features to add to your portfolio project are the following:

* Organize your findings into dictionaries, lists, or another convenient datatype.
* Make predictions about what features are the most influential for an individual’s medical insurance charges based on your analysis.
* Explore areas where the data may include bias and how that would impact potential use cases.

Congrats on completing your portfolio project!
